# Market Basket Analysis
This notebook contains an implementation of A Priori Algorithm. Datasets are taken from :
- https://github.com/jzuniga123/SPS/blob/master/DATA%20624/GroceryDataSet.csv 
- https://www.kaggle.com/shwetabh123/basketballoptimisation

## Loading data and format conversion

In [1]:
import numpy as np
import csv

In [2]:
# read the csv file and store all items in the list "items"
path = 'Market_Basket_Optimisation.csv'
items = [] # list of all items
try:
    f = open(path, 'r')
    rows = csv.reader(f, delimiter=',')
    for row in rows:
        row = [item for item in row if item!=''] # remove redundant entries
        items = items + row
        # print(row)
    f.close()
except:
    print('ERROR: can not found ' + path)
    exit(1)

In [3]:
# remove redundant entries in items
itemset = set(items) # remove duplicate entries
items = list(itemset) # convert back to list
print(items)

['zucchini', 'green beans', 'fresh tuna', 'burger sauce', 'ham', 'black tea', 'tomato sauce', 'eggplant', 'whole wheat rice', 'yogurt cake', 'white wine', 'chili', 'mint', 'blueberries', 'fromage blanc', 'burgers', 'bacon', 'pancakes', 'light mayo', 'hand protein bar', 'cottage cheese', 'chocolate bread', 'red wine', 'shrimp', 'cauliflower', 'whole weat flour', 'strong cheese', 'tomato juice', 'fresh bread', 'spaghetti', 'herb & pepper', 'cider', 'grated cheese', 'chutney', 'babies food', 'energy bar', 'soup', 'tea', 'body spray', 'salt', 'ketchup', 'parmesan cheese', 'hot dogs', 'melons', 'mashed potato', 'bramble', 'candy bars', 'light cream', 'dessert wine', 'frozen smoothie', 'energy drink', 'muffins', 'barbecue sauce', 'ground beef', 'avocado', 'green grapes', 'shallot', 'brownies', 'gluten free bar', 'frozen vegetables', 'french wine', 'napkins', 'pickles', 'pasta', 'bug spray', 'cream', 'oil', 'pet food', 'spinach', 'cookies', 'rice', 'corn', 'olive oil', 'carrots', 'asparagus',

In [4]:
# convert each basket to one-hot encoding
f = open(path, 'r')
rows = csv.reader(f, delimiter=',')
data = [] # matrix to store data
n = len(items) # number of items
for row in rows: # for each customer
    row = [item for item in row if item!='']
    basket = [0]*n # initialize each item to be nonpresent
    for item in row: # for each item
        basket[items.index(item)] = 1 # the item is present
    data.append(basket) # add to data
f.close()
data = np.array(data) # convert to numpy array
# # check if the conversion is correct by inspecting the first basket
# for index in np.argwhere(data[0]>0).flatten():
#     print(items[index])

## Function Definitions

In [5]:
def support(data):
    '''return the support of a 2d array with each row being one data point'''
    m = data.shape[0] # number of data points
    n = data.shape[1] # number of variables
    templist = np.zeros(m)
    for i in range(m): # for each row
        temp = 1
        for j in range(n): # for each column
            temp *= data[i][j]
        templist[i] = temp
    supp = sum(templist)/m
    return supp

In [6]:
def FindHighSupportSubset(data, t):
    '''This function takes in binary numpy array "data" in market basket analysis and the support threshold t, and
    it returns a list "itemsetList" of sets whose k-th position is a set of tuples encoding subsets of itemset 
    with size k with support > t
    '''
    # set up parameters
    numBaskets = data.shape[0] # number of baskets
    numItems   = data.shape[1] # number of items
    items = np.arange(numItems) # indicies of items
    itemsetList = [] # initialize list to return
    
    # compute all size 1 itemsets
    supp = np.sum(data, axis=0)/numBaskets # compute the support for each singleton itemset
    singletons = np.argwhere(supp>0.05).flatten() # store the indicies as an array
    itemsetList.append(set(singletons))
    n0 = len(singletons) # number of qualified singletons
    # compute all size 2 itemsets
    tempset = set()
    for i in range(n0): # for each singleton
        for j in range(i+1, n0): # for each pair of singleton
            supp = support(data[:,[i,j]]) # compute the support
            if supp > t: # if support is larger than the threshold
                tempset = tempset | {(i,j)}   # add the pair to the qualified itemsets
    if len(tempset)!=0: # if the set is not empty
        itemsetList.append(tempset)
    else:
        return itemsetList  # no bigger subsets possible

    # iteratively populate itemsetList  
    for k in range(2,numItems): # for each k corresponding to subset of size k+1
        tempset = set()
        for subsetk in itemsetList[k-1]: # for each subset of size k
            for item in singletons: # for each singleton itemset
                if item not in subsetk: # if singleton not in subset
                    index = list(subsetk) + [item] # join the indices as a list
                    supp = support(data[:, index]) # compute the support
                    # print(index, supp, supp>t)
                    if supp > t: # if support is larger than the threshold
                        index = np.sort(index) # sort the index
                        tempset = tempset | {tuple(index)} # convert to tuple and add to the qualified subsets
        if len(tempset)!=0: # if there is some qualified subset
            itemsetList.append(tempset) # add to list
        else:
            return itemsetList  # no bigger subsets possible
    return itemsetList 

In [7]:
def confidence(data, A, B):
    '''return the confidence of the association rule A=>B, where A and B are sets of indices'''
    return support(data[:,list(A|B)]) / support(data[:,list(B)])

## Apply to dataset

Dataset 'Market_Basket_Optimisation.csv'

In [8]:
itemsetList = FindHighSupportSubset(data, 0.01)
print(itemsetList[1:])

[{(15, 17), (17, 23)}]


In [9]:
# explore
print(items[15], items[17], items[23])
print(confidence(data, {15}, {17}), confidence(data, {17}, {23}), )

burgers pancakes shrimp
0.11079943899018233 0.14738805970149252


Dataset 'GrocerydataSet.csv'

In [10]:
# read the csv file
path = 'GroceryDataSet.csv'
items = [] # list of all items
try:
    f = open(path, 'r')
    rows = csv.reader(f, delimiter=',')
    for row in rows:
        row = [item for item in row if item!='']
        items = items + row
        # print(row)
    f.close()
except:
    print('ERROR: can not found ' + path)
    exit(1)

itemset = set(items) # remove duplicate entries
items = list(itemset) # convert back to list

# convert each basket to one-hot encoding
f = open(path, 'r')
rows = csv.reader(f, delimiter=',')
data = [] # matrix to store data
n = len(items) # number of items
for row in rows: # for each customer
    basket = [0]*n
    row = [item for item in row if item!='']
    for item in row: # for each item
        basket[items.index(item)] = 1 # the item is present
    data.append(basket) # add to data
f.close()
data = np.array(data) # convert to numpy array

In [12]:
itemsetList = FindHighSupportSubset(data, 0.01)
print(itemsetList[1:])

[{(12, 21)}]


In [15]:
print(items[12],'\t', items[21])
print(confidence(data, {12}, {21}))

root vegetables 	 sausage
0.1590909090909091
